In [ ]:
from pathlib import Path

import networkx as nx
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [ ]:
def read_data(dir_name: str, file_name: str):
    """Read the medium articles with lists

    Args:
        dir_name (str): Root directory of the medium title files and lists.

    Returns:
        final_data: merged dataframes with articles and lists
    """

    final_data = pd.read_csv(dir_name+"/"+file_name+".csv")
    return final_data

In [ ]:
final_data = pd.read_csv("data/pipeline_assignment_data/full_data_without_labels.csv")
final_data

In [ ]:
train = read_data("data/pipeline_assignment_data","train")
train

In [ ]:
test = read_data("data/pipeline_assignment_data","test")
test

In [ ]:
train["labels"].value_counts()

In [ ]:
test["labels"].value_counts()

In [ ]:
train.subtitle

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

multilabel_binarizer = LabelEncoder()
multilabel_binarizer.fit(train["labels"])

Y = multilabel_binarizer.transform(train["labels"])
texts = [x[0]+" " + x[1] for x in zip(train.title,train.subtitle)]

In [ ]:
len(list(set(final_data.index)))

In [ ]:
from gensim.models import Word2Vec
all_texts = [x[0]+" " + x[1] for x in zip(final_data.title,final_data.subtitle)]

word2vec_model = Word2Vec([text.split(" ") for text in all_texts], vector_size=128, window=10, epochs=30, sg=1, workers=4,min_count=1)

In [ ]:
Y

In [ ]:
from sklearn.svm import SVC
embeddings = []

for text in texts:
    embeddings.append(np.mean([word2vec_model.wv[word] for word in text.split(" ")], axis=0))
print(np.vstack(embeddings).shape)
X_word2vec = np.vstack(embeddings)


In [ ]:
svc = SVC()
svc.fit(X_word2vec,Y)

In [ ]:
test_embeddings = []
text_texts = [x[0]+" " + x[1] for x in zip(test.title,test.subtitle)]

for text in text_texts:
    test_embeddings.append(np.mean([word2vec_model.wv[word] for word in text.split(" ")], axis=0))
print(np.vstack(test_embeddings).shape)
X_word2vec_test = np.vstack(test_embeddings)


In [ ]:
predictions = svc.predict(X_word2vec_test)

In [ ]:
Y_test = multilabel_binarizer.transform(test["labels"])

Y_test

In [ ]:
from sklearn import metrics
print(metrics.f1_score(Y_test, predictions,average="macro"))


In [ ]:
print(metrics.classification_report(Y_test, predictions))


# Form graph

In [ ]:


from typing import List,Dict


def get_edges(data: pd.DataFrame, nodes) -> List:
    """ Given the dataframe with articles and lists return the set of edges
        Args:
        data (pd.DataFrame): The medium dataset
        nodes (dict): List of nodes
    Returns:
            nodes: dict (nodeid: article title)"""
    edges = []    
    ## START
    
    ##END
    return edges




In [ ]:
def get_nodes(data: pd.DataFrame) -> Dict:
    """ Given the dataframe with articles and lists return the set of nodes
        Args:
        data (pd.DataFrame): The medium dataset
    Returns:
        nodes: dict (nodeid: article title)"""
    nodes = {}
    for index, row in data.iterrows():
        if index not in nodes:
            nodes[index] = row["title"]
    print(len(nodes))
    return nodes

In [ ]:
import networkx as nx
def form_graph(data: pd.DataFrame) -> nx.Graph:
    """Forms graph from medium article dataset.

    Args:
        data (pd.DataFrame): The medium dataset

    Returns:
        G (nx.Graph): The graph.

       """
    texts = [x[0]+" " + x[1] for x in zip(data.title,data.subtitle)]
    nodes = get_nodes(data)
    edges = get_edges(data, nodes)
    graph = nx.Graph()
    graph.add_nodes_from(list(nodes.keys()))
    graph.add_edges_from(edges)
    return graph


In [ ]:
graph = form_graph(final_data)


In [ ]:
len(list(graph.edges()))

# Node2Vec

In [ ]:
# Find isolated nodes with no neighbors before random walks
isolated = [x for x in nx.isolates(graph)]

In [ ]:
def random_walks(G: nx.Graph, num_walks: int, walk_length: int, isolated: List) -> np.ndarray:
    """Perform random walks on the graph.

    Args:
        G (nx.Graph): The graph.
        num_walks (int): The number of random walks for each node.
        walk_length (int): The number of nodes in a random walk.

    Returns:
        np.ndarray: The random walks, shape (n_nodes * num_walks, walk_length)
    """
    result = []
    ### START
    
    ## END
    return np.asarray(result)

In [ ]:
walks = random_walks(graph, 8, 15,isolated)

In [ ]:
from gensim.models import Word2Vec
def fit_node2vec(walks: np.ndarray, vector_size: int, window: int, epochs: int) -> Word2Vec:
    """Train a Node2Vec model on random walks. Uses the GenSim Word2Vec implementation.

    Args:
        walks (np.ndarray): The random walks.
        vector_size (int): Node representation size.
        window (int): Window width.
        epochs (int): Number of epochs.

    Returns:
        Word2Vec: The trained model.
    """
   ### START

In [ ]:
model = fit_node2vec(walks, 128, 5, 10)

In [ ]:
#embeddings = {doc: model.dv[doc] for doc in model.dv.index_to_key}


In [ ]:
def get_nodeids(data: pd.DataFrame):
    nodes = {}
    for index, row in data.iterrows():
        if row["index"] not in nodes:
            nodes[row["index"]] = row["title"]
    return nodes

In [ ]:
train_nodes = get_nodeids(train)
test_nodes = get_nodeids(test)
train_nodes = list(train_nodes.keys())
test_nodes = list(test_nodes.keys())

In [ ]:
len(test_nodes)

In [ ]:
embeddings = {word: model.wv[word] for word in model.wv.index_to_key}


In [ ]:
import scipy
X_train_n2v = np.hstack(
    (
        X_word2vec, np.array([embeddings[str(x)] if x not in isolated else X_word2vec[idx] for idx, x in enumerate(train_nodes)  ], dtype=np.float32),

    )
)
X_test_n2v =np.hstack(
    (
        X_word2vec_test,
        np.array([embeddings[str(x)] if x not in isolated else X_word2vec_test[idx] for idx, x in enumerate(test_nodes)  ], dtype=np.float32),
    )
)

In [ ]:
X_test_n2v.shape,X_train_n2v.shape

In [ ]:
X_train_n2v.shape

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC,SVC

svc = SVC()
svc.fit(X_train_n2v,Y)

In [ ]:
Y.shape

In [ ]:
X_test_n2v.shape

In [ ]:
predictions = svc.predict(X_test_n2v)

In [ ]:
predictions

In [ ]:
from sklearn import metrics
print(metrics.f1_score(Y_test, predictions,average="macro"))


In [ ]:
print(metrics.classification_report(Y_test, predictions))


# Only node2vec features

In [ ]:
import scipy
X_train_n2v =  np.array([embeddings[str(x)] if x not in isolated else np.zeros((128)) for idx, x in enumerate(train_nodes)  ], dtype=np.float32)

X_test_n2v =np.array([embeddings[str(x)] if x not in isolated else np.zeros((128)) for idx, x in enumerate(test_nodes)  ], dtype=np.float32)
 

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC,SVC

svc = SVC()
svc.fit(X_train_n2v,Y)

In [ ]:
predictions = svc.predict(X_test_n2v)

In [ ]:
from sklearn import metrics
print(metrics.f1_score(Y_test, predictions,average="macro"))


In [ ]:
print(metrics.classification_report(Y_test, predictions))
